# Plasma GAN
Andrew E. Davidson  
aedavids@ucsc.edu  
8/29/24  

Copyright (c) 2020-2023, Regents of the University of California All rights reserved. https://polyformproject.org/licenses/noncommercial/1.0.0  

ref:
- boiler plate and gene expression data load
    * intraExtraRNA_POC/jupyterNotebooks/elife/randomForestGTEx_TCGAGeneSignature.ipynb
      + aedwip what did this do
    * deconvolutionAnalysis/jupyterNotebooks/randomForestGeneSignatureDeconvolution.ipynb
      + trained a random forest model on the GTEx_TCGA training data set. It has perfect TP

## TODO:


the random forest on GTEX_TCGA had 712 genes

 so we have a list of 712 HUGO Genes, we need to map these to elife

elifeutilities.py

206 def loadElifeTrainingData
    123 def loadCounts
    236 loadMetaData

    assume we have a list of hugo genes
    282 selectFeatures
    284 fixBest10CuratedDegree1_ce467ff
    
```

In [1]:
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display
import joblib
import logging
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# from sklearn.metrics import confusion_matrix
from sklearn.ensemble      import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# from sklearn.metrics         import recall_score
# from sklearn.metrics         import roc_auc_score
# from sklearn.metrics         import make_scorer

import sys

/data/tmp/ipykernel_3802778/3098827269.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
notebookName = ipynbname.name()
notebookPath = ipynbname.path()
notebookDir = os.path.dirname(notebookPath)

outDir = f'{notebookDir}/{notebookName}.out'
#outDir = f'/private/groups/kimlab/aedavids/deconvolution/{notebookName}.out'
os.makedirs(outDir, exist_ok=True)
print(f'outDir:\n{outDir}')

modelOutDir = os.path.join(outDir, "model")
os.makedirs(modelOutDir, exist_ok=True)
print(f'\nmodelOutDir ;\n{modelOutDir}')

imgOut = f'{outDir}/img'
os.makedirs(imgOut, exist_ok=True)
print(f'\nimgOut :\n{imgOut}')

#loglevel = "INFO"
loglevel = "WARN"
# logFMT = "%(asctime)s %(levelname)s [thr:%(threadName)s %(name)s %(funcName)s() line:%(lineno)s] [%(message)s]"
logFMT = "%(asctime)s %(levelname)s %(name)s %(funcName)s() line:%(lineno)s] [%(message)s]"
logging.basicConfig(format=logFMT, level=loglevel)    
logger = logging.getLogger(notebookName)

outDir:
/private/home/aedavids/extraCellularRNA/intraExtraRNA_POC/jupyterNotebooks/elife/gan/plasmaGAN.out

modelOutDir ;
/private/home/aedavids/extraCellularRNA/intraExtraRNA_POC/jupyterNotebooks/elife/gan/plasmaGAN.out/model

imgOut :
/private/home/aedavids/extraCellularRNA/intraExtraRNA_POC/jupyterNotebooks/elife/gan/plasmaGAN.out/img


In [3]:
# setting the python path allows us to run python scripts from using
# the CLI. 
PYTHONPATH = os.environ['PYTHONPATH']
print("ORIG_PYTHONPATH: {}\n".format(PYTHONPATH))

gitRepoRoot = !git rev-parse --show-toplevel
gitRepoRoot = gitRepoRoot[0]

#
# add deconvolutionAnalysis modules
#
deconvolutionModules = f'{gitRepoRoot}/deconvolutionAnalysis/python'
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = PYTHONPATH + f':{deconvolutionModules}'
#print("PYTHONPATH: {}\n".format(PYTHONPATH))

sys.path.append( str(deconvolutionModules) )
#print("\nsys.path:\n{}\n".format(sys.path))

#
# add intraExtraRNA_POC modules
#
intraExtraRNA_POCModules = f'{gitRepoRoot}/intraExtraRNA_POC/python/src'
print("intraExtraRNA_POCModules: {}\n".format(intraExtraRNA_POCModules))

PYTHONPATH = PYTHONPATH + f':{intraExtraRNA_POCModules}'
#print("PYTHONPATH: {}\n".format(PYTHONPATH))

sys.path.append( str(intraExtraRNA_POCModules) )
#print("\nsys.path:\n{}\n".format(sys.path))

ORIG_PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/python

intraExtraRNA_POCModules: /private/home/aedavids/extraCellularRNA/intraExtraRNA_POC/python/src



In [4]:
# local imports
# from analysis.utilities import saveList
from analysis.utilities import loadList
# from intraExtraRNA.elifeUtilities import loadElifeTrainingData
# from intraExtraRNA.elifeUtilities import validElifeCategories

from intraExtraRNA.elifeUtilities import loadCounts
from intraExtraRNA.elifeUtilities import loadMetaData
from intraExtraRNA.elifeUtilities import selectFeatures

## Find features used to train randomForestGeneSignatureDeconvolution model
This model was trained on  GTEx_TCGA training data set has perfect TP!

ref: deconvolutionAnalysis/jupyterNotebooks/randomForestGeneSignatureDeconvolution

model saved to /private/groups/kimlab/aedavids/deconvolution/randomForestGeneSignatureDeconvolution.out

best10CuratedDegree1_ce467ff_elife.orderedFeatures.txt has the list of features. The "elife" token in the file name is a little confusing. The CIBERSORTx signature matrix for best10CuratedDegree1_ce467ff has a couple of gene that we could not map to the elife data set. This list of features is the elife subset of features

In [5]:
def getListOfFeatures() :
    '''
    wrapper function to prevent global namespace polution
    '''
    rfPath = "/private/groups/kimlab/aedavids/deconvolution/randomForestGeneSignatureDeconvolution.out/model"
    fileName = "best10CuratedDegree1_ce467ff_elife.orderedFeatures.txt"
    p = rfPath + "/" + fileName
    print(f'loading features:\n{p}' )
    ret = loadList( p )

    with open(p, "r") as f: 
        dataStr = f.read() 
    
    return ret

HUGO_featuresNames = getListOfFeatures()

print(f'\nnumber of HUGO_featuresNames : {len(HUGO_featuresNames)}')
print( f'first 3: {HUGO_featuresNames[:3]} last 3: {HUGO_featuresNames[-3:]}' )

loading features:
/private/groups/kimlab/aedavids/deconvolution/randomForestGeneSignatureDeconvolution.out/model/best10CuratedDegree1_ce467ff_elife.orderedFeatures.txt

number of HUGO_featuresNames : 712
first 3: ['(GGTG)n', '(GT)n', '(TA)n'] last 3: ['ZRANB1', 'ZSWIM4', 'ZYG11B']


# Load Gene Expression Data

In [6]:
# 206 def loadElifeTrainingData
#     123 def loadCounts
#     236 loadMetaData

#     assume we have a list of hugo genes
#     282 selectFeatures
#     284 fixBest10CuratedDegree1_ce467ff

In [7]:
transposedCountsDF = loadCounts()
print( f'transposedCountsDF.shape : {transposedCountsDF.shape}' )
transposedCountsDF.iloc[0:5, 0:5]

transposedCountsDF.shape : (224, 76555)


gene,(A)n,(AAA)n,(AAAAAAC)n,(AAAAAAG)n,(AAAAAAT)n
SRR14506659,201.672053,0.0,0.0,0.0,0.0
SRR14506660,110.450773,0.0,0.0,0.0,0.0
SRR14506661,3722.776395,0.0,0.0,0.0,0.0
SRR14506662,1394.605651,0.0,0.0,0.0,0.0
SRR14506663,2843.473730,0.0,0.0,0.0,0.0


In [8]:
metaDF = loadMetaData()
print( f'metaDF.shape : {metaDF.shape}' )
metaDF.head()

metaDF.shape : (224, 2)


,sample_id,diagnosis
0,SRR14506659,Esophagus Cancer
1,SRR14506660,Esophagus Cancer
2,SRR14506661,Esophagus Cancer
3,SRR14506662,Esophagus Cancer
4,SRR14506663,Esophagus Cancer


In [9]:
%%time
elifeLungGenes, missingElifeGenes, mapDF = selectFeatures( HUGO_featuresNames )

2024-10-09 16:45:07,065 WARNING intraExtraRNA.elifeUtilities selectFeatures() line:420] [missingInV39Set: {'LTR106'}]


CPU times: user 1min 14s, sys: 16.4 s, total: 1min 30s
Wall time: 1min 32s


In [10]:
print( f'len(elifeLungGenes) {len(elifeLungGenes)}' )
print( f'first 3 {elifeLungGenes[:3]} last 3 {elifeLungGenes[-3:]}' )

len(elifeLungGenes) 713
first 3 ['ENSG00000198744.5', 'ENSG00000187642.9', 'ENSG00000215915.10'] last 3 ['L1MEa', 'L1M2a1', 'LTR106_Mam']


In [11]:
assert len(missingElifeGenes) == 0,"ERROR"

In [12]:
print( f'mapDF.shape : {mapDF.shape}' )
mapDF.head()

mapDF.shape : (715, 3)


,HUGO_v35,ENSG_v35,ENSG_v39
0,MTCO3P12,ENSG00000198744.5,ENSG00000198744.5
1,PERM1,ENSG00000187642.9,ENSG00000187642.9
2,ATAD3C,ENSG00000215915.10,ENSG00000215915.10
3,SMIM1,ENSG00000235169.10,ENSG00000235169.11
4,ANGPTL7,ENSG00000171819.5,ENSG00000171819.5


In [ ]:
select genes and cluster

In [13]:
pipelineStageName = "best10CuratedDegree1_ce467ff"


In [14]:
# example from http://localhost:13892/lab/tree/intraExtraRNA_POC/jupyterNotebooks/elife/elifeBinaryRandomForestResults.ipynb
# wrong example. we want something like 700 features not best 10 for lung. or best 10 for liver, ..
#
# %%time
# pipelineStageName = "best10CuratedDegree1_ce467ff"
# selectElifeCategories = ["Colorectal Cancer", "Esophagus Cancer", "Healthy donor", "Liver Cancer", "Lung Cancer", "Stomach Cancer"]
# t = loadElifeTrainingData(pipelineStageName,
#                              features, 
#                              selectElifeCategories,
#                              )

# HUGO_Genes, elifeGenes, missingGenes, countDF, metaDF, XDF, yNP, labelEncoderElife, mapDF = t

# print(f'missingGenes:\n{missingGenes}')